In [3]:
# !pip install pandas
# !pip install matplotlib
# !pip install numpy
!pip install openpyxl

In [4]:
import pandas as pd
from py3dbp import Packer, Bin, Item
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from collections import defaultdict
from IPython.display import display, HTML
import base64
from io import BytesIO
# Load the Excel file
file_path = 'Optimization Problem.xlsx'
packages_df = pd.read_excel(file_path, sheet_name='Packages')
cartons_df = pd.read_excel(file_path, sheet_name='Cartons')
# Function to generate a random color
def get_random_color():
    return np.random.rand(3,)
# Function to add a 3D box (representing an item) without labels
def add_box(ax, item, color):
    # Extracting position and dimensions
    pos = np.array(item.position, dtype=float)
    dim = np.array(item.get_dimension(), dtype=float)
    # Create a rectangular prism
    xx, yy = np.meshgrid([pos[0], pos[0]+dim[0]], [pos[1], pos[1]+dim[1]])
    ax.plot_surface(xx, yy, np.full_like(xx, pos[2]), color=color, alpha=0.5, edgecolor='k', linewidth=0.5)
    ax.plot_surface(xx, yy, np.full_like(xx, pos[2]+dim[2]), color=color, alpha=0.5, edgecolor='k', linewidth=0.5)
    
    yy, zz = np.meshgrid([pos[1], pos[1]+dim[1]], [pos[2], pos[2]+dim[2]])
    ax.plot_surface(np.full_like(yy, pos[0]), yy, zz, color=color, alpha=0.5, edgecolor='k', linewidth=0.5)
    ax.plot_surface(np.full_like(yy, pos[0]+dim[0]), yy, zz, color=color, alpha=0.5, edgecolor='k', linewidth=0.5)
    
    xx, zz = np.meshgrid([pos[0], pos[0]+dim[0]], [pos[2], pos[2]+dim[2]])
    ax.plot_surface(xx, np.full_like(xx, pos[1]), zz, color=color, alpha=0.5, edgecolor='k', linewidth=0.5)
    ax.plot_surface(xx, np.full_like(xx, pos[1]+dim[1]), zz, color=color, alpha=0.5, edgecolor='k', linewidth=0.5)
# Get user input for the package to pack
use_custom_dimensions = input("Do you want to use custom package dimensions? (yes/no): ").strip().lower()
if use_custom_dimensions == "yes":
    package_id = "CustomPackage"
    length = float(input("Enter the package length (in inches): "))
    width = float(input("Enter the package width (in inches): "))
    height = float(input("Enter the package height (in inches): "))
else:
    package_id = input("Enter the Package ID to pack: ")
    selected_package = packages_df[packages_df['Package_ID'] == package_id]
    if selected_package.empty:
        print("Package ID not found.")
        exit()
    package = selected_package.iloc[0]
    length = package['PKG_LNGTH_IN']
    width = package['PKG_WIDTH_IN']
    height = package['PKG_DEPTH_IN']
# Define the item with the dimensions from the selected package
item_data = {
    "name": package_id, 
    "length": length, 
    "width": width, 
    "height": height, 
    "weight": 0
}
# Initialize variables to track the best fit
best_fit_container = None
best_fit_volume_utilized_percentage = 0
# Initialize HTML content
html_content = """
<!DOCTYPE html>
<html>
<head>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            background-color: #f5f5f5;
            color: #333333;
            margin: 0;
            padding: 20px;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #ffffff;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            border-radius: 8px;
        }
        h1 {
            color: #004080;
            text-align: center;
            font-size: 24px;
            margin-bottom: 20px;
        }
        h2 {
            color: #0056b3;
            border-bottom: 2px solid #0056b3;
            padding-bottom: 10px;
            font-size: 20px;
            margin-top: 40px;
        }
        .container-info {
            font-weight: bold;
            color: #006600;
            margin-top: 10px;
            font-size: 16px;
        }
        .instructions {
            margin-top: 20px;
        }
        .instructions h3 {
            color: #e69500;
            font-size: 18px;
        }
        .instructions ul {
            list-style-type: none;
            padding-left: 0;
        }
        .instructions ul li {
            background: #f8f9fa;
            padding: 10px;
            margin-bottom: 5px;
            border-radius: 5px;
        }
        .plot img {
            display: block;
            max-width: 100%;
            height: auto;
            margin: 20px 0;
            border: 1px solid #ddd;
            border-radius: 5px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .best-fit {
            font-weight: bold;
            color: #cc3300;
            margin-top: 20px;
            font-size: 16px;
        }
        .footer {
            text-align: center;
            margin-top: 20px;
            color: #6c757d;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Packing Optimization Report</h1>
"""
# Loop through each container
for index, carton in cartons_df.iterrows():
    # Define your storage unit as a 'Bin'
    storage_unit = Bin(carton['Description'], carton['ID Length (in)'], carton['ID Width (in)'], carton['ID Height (in)'], 1)
    # Initialize the Packer
    packer = Packer()
    # Add the storage unit to the packer
    packer.add_bin(storage_unit)
    # Add items in batches
    batch_size = 100
    num_batches = 10  # 1000 items in total
    for i in range(num_batches):
        batch_items = [Item(item_data["name"], item_data["length"], item_data["width"], item_data["height"], item_data["weight"]) for _ in range(batch_size)]
        for item in batch_items:
            packer.add_item(item)
    # Run the packing algorithm
    packer.pack()
    # Calculate the volume of the storage unit
    storage_volume = float(storage_unit.width * storage_unit.height * storage_unit.depth)
    # Calculate the volume of a single item
    item_volume = float(item_data["length"] * item_data["width"] * item_data["height"])
    # Calculate the total volume utilized and the number of items fit
    total_items_fit = sum(len(b.items) for b in packer.bins)
    total_volume_utilized = float(total_items_fit * item_volume)
    # Calculate the percentage of volume utilized
    volume_utilized_percentage = (total_volume_utilized / storage_volume) * 100
    # Check if this container is the best fit
    if volume_utilized_percentage > best_fit_volume_utilized_percentage:
        best_fit_container = carton
        best_fit_volume_utilized_percentage = volume_utilized_percentage
    # Collect packing instructions
    instructions = defaultdict(int)
    for b in packer.bins:
        for item in b.items:
            pos = tuple(item.position)
            dim = tuple(item.get_dimension())
            key = (pos, dim)
            instructions[key] += 1
    # Generate HTML content for this container
    container_html = f"""
        <h2>Container: {carton['Description']} ({carton['ID Length (in)']} x {carton['ID Width (in)']} x {carton['ID Height (in)']})</h2>
        <div class="container-info">Package: {package_id} ({item_data['length']} x {item_data['width']} x {item_data['height']})</div>
        <div class="container-info">Total number of items fit: {total_items_fit}</div>
        <div class="container-info">Percentage of volume utilized: {volume_utilized_percentage:.2f}%</div>
        <div class="instructions">
            <h3>Packing Instructions:</h3>
            <ul>
    """
    for (pos, dim), count in instructions.items():
        container_html += f"<li>{count} items at position (x, y, z): ({pos[0]}, {pos[1]}, {pos[2]}) with dimensions (L x W x H): ({dim[0]} x {dim[1]} x {dim[2]})</li>"
    container_html += """
            </ul>
        </div>
    """
    html_content += container_html
    # Create a 3D plot
    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection='3d')
    # Adding each item in the packed bins to the plot
    for b in packer.bins:
        for item in b.items:
            color = get_random_color()  # Get a random color for each item
            add_box(ax, item, color)
    # Setting the limits to match the storage unit size
    ax.set_xlim([0, carton['ID Length (in)']])
    ax.set_ylim([0, carton['ID Width (in)']])
    ax.set_zlim([0, carton['ID Height (in)']])
    # Setting the aspect ratio to match the storage unit dimensions
    ax.set_box_aspect([carton['ID Length (in)'], carton['ID Width (in)'], carton['ID Height (in)']])
    # Labels and title
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title(f'3D Visualization of Items (SKU: {item_data["name"]}) in {carton["Description"]} ({carton["ID Length (in)"]} x {carton["ID Width (in)"]} x {carton["ID Height (in)"]})')
    # Adjust layout to ensure title is not cut off
    plt.tight_layout(pad=2.0)
    # Save plot to a BytesIO object
    img_bytes = BytesIO()
    plt.savefig(img_bytes, format='png', bbox_inches='tight')
    img_bytes.seek(0)
    img_base64 = base64.b64encode(img_bytes.read()).decode('utf-8')
    # Embed the image in HTML
    html_content += f'<div class="plot"><img src="data:image/png;base64,{img_base64}" alt="3D Visualization"></div>'
    plt.close()
# Add best fit container info to HTML
if best_fit_container is not None:
    html_content += f'<div class="best-fit">The best fit container is {best_fit_container["Description"]} with a volume utilization of {best_fit_volume_utilized_percentage:.2f}%</div>'
else:
    html_content += '<div class="best-fit">No suitable container found.</div>'
# Close HTML content
html_content += """
    </div>
    <div class="footer">
        &copy; 2024 Packing Optimization Report
    </div>
</body>
</html>
"""
# Save the HTML content to a file
report_filename = 'packing_report.html'
with open(report_filename, 'w') as file:
    file.write(html_content)
# Display a link to download the report
display(HTML(f'<a href="{report_filename}" download>Download Packing Report</a>'))